In [1]:
import glob, os, yaml, feather
import numpy as np
import pandas as pd
from os.path import isfile, join

import csv
import pprint

In [2]:
# questionnaires

# compute results
# for questionnaires where results are the average of all items

def compute_quest_average(quest):
    try:
        quest_results = 0
        i = 0
        for item in quest: 
            if item['receivedAnswer'] == None:
                break
            quest_results += int(item['receivedAnswer'])
            i += 1
        return quest_results / i
    except ZeroDivisionError:
        return None

# compute results TA-EG
# subscale enthusiasm
def compute_taeg_subscale_enthusiasm(quest): 
    quest_results = 0 
    enthusiasm = [0, 2, 7, 9, 14]
    for i in enthusiasm:
        if quest[i]['receivedAnswer'] == None:
            break
        quest_results +=int(quest[i]['receivedAnswer'])
    return quest_results / len(enthusiasm)

#subscale competence
def compute_taeg_subscale_competence(quest): 
    quest_results = 0 
    competence = [13, 16, 17]
    competence_rev = [3]
    for i in competence:
        if quest[i]['receivedAnswer'] == None:
            break
        quest_results +=int(quest[i]['receivedAnswer'])
    for i in competence_rev:
        if quest[i]['receivedAnswer'] == None:
            break
        quest_results += (6-int(quest[i]['receivedAnswer']))
    return quest_results / (len(competence) + len(competence_rev))

# subscale positive attitude
def compute_taeg_subscale_pos_attitude(quest): 
    quest_results = 0 
    pos_attitude = [4, 8, 10, 11, 18]
    for i in pos_attitude:
        if quest[i]['receivedAnswer'] == None:
            break
        quest_results +=int(quest[i]['receivedAnswer'])
    return quest_results / len(pos_attitude)

# subscale negative attitude 
def compute_taeg_subscale_neg_attitude(quest): 
    quest_results = 0 
    neg_attitude = [1, 5, 6, 12, 15]
    for i in neg_attitude:
        if quest[i]['receivedAnswer'] == None:
            break
        quest_results +=int(quest[i]['receivedAnswer'])
    return quest_results / len(neg_attitude)

def compute_taeg(quest):
    quest_results = 0 
    pos_items = [0, 2, 4, 7, 8, 9, 10, 11, 13, 14, 16, 17, 18]
    neg_items = [1, 3, 5, 6, 12, 15]
    for i in pos_items:
        if quest[i]['receivedAnswer'] == None:
            break
        quest_results +=int(quest[i]['receivedAnswer'])
    for i in neg_items:
        if quest[i]['receivedAnswer'] == None:
            break
        quest_results += (6-int(quest[i]['receivedAnswer']))
    return quest_results / (len(pos_items) + len(neg_items))
    
# get taeg values
def get_taeg_values():
    taeg_values = np.array(list(filter(lambda userInputs: userInputs['category'] == 'Tech Usage TAEG' , userInputs)))[:19]
    return taeg_values

# get comments on taeg
def get_taeg_comments():
    taeg_comments = np.array(list(filter(lambda userInputs: userInputs['category'] == 'Tech Usage TAEG' , userInputs)))[19]
    
    return taeg_comments['receivedAnswer']

    
#get nasa-tlx values

def get_nasa_tlx_values(version):
    nasa_tlx_values = np.array(list(filter(lambda userInputs: userInputs['category'] == version , userInputs)))[:6]
    return nasa_tlx_values



In [3]:
# get tech usage
def get_tech_usage():
    tech_usage_values = np.array(list(filter(lambda userInputs: userInputs['category'] == 'Tech Usage TAEG' , userInputs)))[:5]
    
    

In [4]:
##  SUS


# get sus values
def get_sus_values():
    sus_values = np.array(list(filter(lambda userInputs: userInputs['category'] == 'UX Questionnaire' , userInputs)))[:10]
    return sus_values


# compute sus score
def compute_sus(quest):
    quest_results = 0 
    pos_items = [0, 2, 4, 6, 8]
    neg_items = [1, 3, 5, 7]
    for i in pos_items:
        if quest[i]['receivedAnswer'] == None:
            break
        quest_results +=(int(quest[i]['receivedAnswer']) -1)
    for i in neg_items:
        if quest[i]['receivedAnswer'] == None:
            break
        quest_results += (5-int(quest[i]['receivedAnswer']))
    return quest_results * 2.5

In [5]:
# time until purposeful action after interruption 

# find index for certain dictionaries in list (one condition)
def find(lst, key, value):
    for dic in lst:
        if dic[key] == value:
            return lst.index(dic)
# interuption 

def compute_purposeful_action_lag(version):
    # find time out for wiki task
    try:
        time_out_wiki_index = find(events, 'description', version)

        time_out_wiki_time = int(events[time_out_wiki_index]['timestamp'])


    # find time when 'Bestellliste' is loaded after wiki 1
        end_of_interruption_event = next(event for event in events if (event['type'] == "OrderListFocus" and int(event['timestamp']) > time_out_wiki_time))
        end_of_interruption_time = int(end_of_interruption_event['timestamp'])

        print(end_of_interruption_event)
        
    # find time when resarch task times out
        research_task_time_out_index = next(event for event in events if (event['description'].startswith('timeout for Research Task') and int(event['timestamp']) > time_out_wiki_time))
        research_task_time_out = int(research_task_time_out_index['timestamp'])  
    
    # find next meaningful user action
    

        try:
            next_user_action_event = next(event for event in events if ((event['type'] == 'OrderListUserInput' or ((event['type'] == 'UserAction') and not (event['description'].startswith('user clicked Website in sidebar')))) and (int(event['timestamp']) > end_of_interruption_time)))
            next_user_action_time = int(next_user_action_event['timestamp'])
            if next_user_action_time >  research_task_time_out:
                print('No meaningful action during this task.')
                return None
            else:
                print(next_user_action_event)    
    
    # compute time between the moment that "Bestellliste" is shown and next user action
                purposeful_action_lag = next_user_action_time - end_of_interruption_time
    
                return purposeful_action_lag
        except StopIteration:
            print('No meaningful action during this task.')
            return None
    
    except TypeError:
        return None


In [6]:
# resumption lag


# interuption 

def compute_resumption_lag(version):
    # find time out for wiki 
    try:
        time_out_wiki_index = find(events, 'description', version)

        time_out_wiki_time = int(events[time_out_wiki_index]['timestamp'])


    # find time when 'Bestellliste' is loaded after wiki task times out
        end_of_interruption_event = next(event for event in events if (event['type'] == "OrderListFocus" and int(event['timestamp']) > time_out_wiki_time))
        end_of_interruption_time = int(end_of_interruption_event['timestamp'])

        
    # find time when resaarch task times out
        research_task_time_out_index = next(event for event in events if (event['description'].startswith('timeout for Research Task') and int(event['timestamp']) > time_out_wiki_time))
        research_task_time_out = int(research_task_time_out_index['timestamp'])  
    
    # find next user action
    
        next_event = next(event for event in events if ((event['type'] == 'OrderListUserInput' or event['type'] == 'UserAction' or event['type'] == 'WebContentServer') and (int(event['timestamp']) > end_of_interruption_time)))
        next_time = int(next_event['timestamp'])
        if next_time > research_task_time_out:
            print('User did nothing during this task.')
            return None
        else:
    # compute time between the moment that "Bestellliste" is shown and next user action
            resumption_lag = next_time - end_of_interruption_time
            return resumption_lag
    
    except TypeError:
        return None


In [7]:
# List input lags
def compute_order_list_time(version):
    try:
        time_out_wiki_index = find(events, 'description', version)

        time_out_wiki_time = int(events[time_out_wiki_index]['timestamp'])


    # find time when 'Bestellliste' is loaded after wiki task times out
        end_of_interruption_event = next(event for event in events if (event['type'] == "OrderListFocus" and int(event['timestamp']) > time_out_wiki_time))
        end_of_interruption_time = int(end_of_interruption_event['timestamp'])

        
    # find time when resarch task times out
        research_task_time_out_index = next(event for event in events if (event['description'].startswith('timeout for Research Task') and int(event['timestamp']) > time_out_wiki_time))
        research_task_time_out = int(research_task_time_out_index['timestamp'])  
        
    # find next order-list-input
        
        try: 
            next_list_input = next(event for event in events if ((event['type'] == 'OrderListUserInput') and (int(event['timestamp']) > end_of_interruption_time)))
            next_list_input_time = int(next_list_input['timestamp'])
            if next_list_input_time >  research_task_time_out:
                print('No order list input during this task.')
                return None
            else:
                print(next_list_input)

    
    
    # compute time between the moment that "Bestellliste" is shown and next user action
                order_list_input_lag = next_list_input_time - end_of_interruption_time
    
                return order_list_input_lag
    
        except StopIteration:
            print('No order list input during this task.')
            return None
    
    except TypeError:
        return None


In [8]:
# average times between clicks (last two minutes of warm up task)

def get_average_click_intervall(): 
    # find moment of first interruption
    try: 
        first_interruption_index = find(events, 'description', 'timeout for Research Task 1 triggered')
        first_interruption_time = int(events[first_interruption_index]['timestamp'])
        
        #find moment two minutes before
        t1 = first_interruption_time - 120000
        
        # count how many clicks the user did in this time invervall
        clicks_nr = sum((event['type'] == 'OrderListUserInput' or event['type'] == 'UserAction' or event['type'] == 'WebContentServer') and (t1 < int(event['timestamp']) < first_interruption_time) for event in events)
        print(clicks_nr)
        # compute average time between clicks
        average_click_intervall = 120000 / clicks_nr 
        print(average_click_intervall)
        return average_click_intervall
    except TypeError: 
        return None
        

In [9]:
# get answers to tech usage question
def get_tech_usage_MC():
    tech_usage= np.array(list(filter(lambda userInputs: userInputs['category'] == 'Tech Usage MC' , userInputs)))[:4]
    #possible_answers = ['Technologienutzung: Arbeiten mit Office Programmen (Word, Excel, Powerpoint, etc.)', 'Technologienutzung: studienbezogene Aufgaben (z.B. Vorlesungen nachbereiten, Hausarbeiten schreiben, Literaturrecherche, etc.)', 'Technologienutzung: Email / im Internet surfen (z.B. Facebook, Spiegel Online, etc.)', 'Technologienutzung: Programmieren']
    received_answers_tech_usage = list(map(lambda tech_usage: tech_usage['receivedAnswer'] == 'true', tech_usage))
    return received_answers_tech_usage
    
def get_tech_usage_MC_others():
    tech_usage_others = np.array(list(filter(lambda userInputs: userInputs['category'] == 'Tech Usage MC' , userInputs)))[4]
    return tech_usage_others['receivedAnswer']

In [10]:
# Sickness replacement task

def get_replacement_task_time():
    try:
        replacement_task_end = events[find(events, 'description', 'user submitted Vertretungs-Task, now showing UX questionnaire')]['timestamp']
        replacements_task_start = events[find(events, 'description', 'got instruction acknowledgement of phase VERTRETUNG_INTRO')]['timestamp']
    
        return int(replacement_task_end) - int(replacements_task_start)
    except TypeError:
        return None

In [11]:
## usage of the prototype


df_useractions = pd.DataFrame([], columns = ['add_to_context', 'remove_from_context', 'highlights', 'open_website_from_sidebar', 'add_note', 'remove_note', 'remove_activity'])
df_u = pd.DataFrame([], columns = ['add_to_context', 'remove_from_context', 'highlights', 'open_website_from_sidebar', 'add_note', 'remove_note', 'remove_activity'])

directory = "C:\\Users\\janin\\Desktop\\Studium\\Masterarbeit\\03_Erhebung"
os.chdir(directory)
for file in os.listdir(directory):
    if file.endswith(".log"):
        with open(file, newline = '', encoding="utf-8") as stream:
            data_reader = csv.DictReader(stream, delimiter='\t')
            data_list = []
            for row in data_reader:
                data_list.append(dict(row))
            filename = os.path.splitext(file)[0]

            values = list(data_list[1])
            value_1 = values[0]
            value_2 = values[1]
            value_3 = values[2]

            for dictionary in data_list: 
                dictionary['type'] = dictionary.pop(value_2)
                dictionary['description'] = dictionary.pop(value_3)
                dictionary['timestamp'] = dictionary.pop(value_1)

            ## events is a list of dictionaries    
            events = [{'type': value_2, 'description': value_3, 'timestamp': value_1}] + data_list



            ## find start and end of working on research task


            # tutorial start
            tutorial_start_index = find(events, 'description', 'now showing instructions of TUTORIAL_AND_1ST_MAIN')
            tutorial_start_time = int(events[tutorial_start_index]['timestamp'])

            # start of warm up task (trial 0)
            warmup_start_index = find(events, 'description', 'Research Task 1 started')
            warmup_start_time = int(events[warmup_start_index]['timestamp'])

            # end of warm up task (trial 0)
            first_interruption_index = find(events, 'description', 'timeout for Research Task 1 triggered')
            first_interruption_time = int(events[first_interruption_index]['timestamp'])

            # resumption 1
            res_1_index = find(events, 'description', 'timeout for Wiki Task 1 triggered')
            res_1_time = int(events[res_1_index]['timestamp'])

            # end of trial 1
            second_interruption_index = find(events, 'description', 'timeout for Research Task 2 triggered')
            second_interruption_time = int(events[second_interruption_index]['timestamp'])

            # resumption 2
            res_2_index = find(events, 'description', 'timeout for Wiki Task 2 triggered')
            res_2_time = int(events[res_2_index]['timestamp'])

            # end of trial 2
            third_interruption_index = find(events, 'description', 'timeout for Research Task 3 triggered')
            third_interruption_time = int(events[third_interruption_index]['timestamp'])

            # resumption 3
            res_3_index = find(events, 'description', 'timeout for Wiki Task 3 triggered')
            res_3_time = int(events[res_3_index]['timestamp'])

            # end of trial 3 (last trial)
            fourth_interruption_index = find(events, 'description', 'timeout for Research Task 4 triggered')
            fourth_interruption_time = int(events[fourth_interruption_index]['timestamp'])
            
            i = 0
            trials = ["tutorial", "warm up", "trial 1", "trial 2", "trial 3"]
            for t1, t2 in [[tutorial_start_time, warmup_start_time], [warmup_start_time, first_interruption_time], [res_1_time, second_interruption_time], [res_2_time, third_interruption_time], [res_3_time, fourth_interruption_time]]: 

            ## get all useractions
                useractions = list(filter(lambda events: events['type'] == 'UserAction' and int(t1) < int(events['timestamp']) and int(events['timestamp']) < int(t2), events))
            ## only descriptions
                useractions_descriptions = [d['description'] for d in useractions]

            ## user added webpage to context
                add_to_context = list(filter(lambda element: element.startswith("user added webpage to context:"), useractions_descriptions))   

            ## remove webpage from context
                remove_from_context = list(filter(lambda element: element.startswith("user removed webpage from context:"), useractions_descriptions))

            # user highlighted
                highlights = list(filter(lambda element: element.startswith("user highlighted:"), useractions_descriptions))

            ## open website from sidebar
                open_website_from_sidebar = list(filter(lambda element: element.startswith("user clicked Website in sidebar:"), useractions_descriptions))

            ## add note to sidebar
                add_note = list(filter(lambda element: element.startswith("user added note to webpage"), useractions_descriptions))

            ## remove note
                remove_note = list(filter(lambda element: element.startswith("user removed note"), useractions_descriptions))

            ## remove activity
                remove_activity = list(filter(lambda element: element.startswith("user removed activity from context:"), useractions_descriptions))
                
            ## interesting actions
                interesting_actions = add_to_context + remove_from_context + highlights + open_website_from_sidebar + add_note + remove_note + remove_activity
            ## remaining actions
                rest = list(set(useractions_descriptions) - set(interesting_actions))

                nr_of_actions = [trials[i]] + list(map(len, [add_to_context, remove_from_context, highlights, open_website_from_sidebar, add_note, remove_note, remove_activity]))

                df_u = pd.DataFrame([nr_of_actions], columns = ['trial', 'add_to_context', 'remove_from_context', 'highlights', 'open_website_from_sidebar', 'add_note', 'remove_note', 'remove_activity'])
                
                i += 1
                
                with open(filename + '.yaml', 'r') as stream:

                    background_and_quest = yaml.safe_load(stream)
                    userInputs = background_and_quest['userInputs']

                    df_0 = pd.DataFrame([[int(background_and_quest['participantNumber']),  float(background_and_quest['version'])]], columns= ['partnr', 'version'])
                    df_u_0 = pd.concat([df_0, df_u], axis=1, sort=False)

                    df_useractions = df_useractions.append(df_u_0, ignore_index=True)



C:\Users\janin\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [12]:
## get performance before the first interruption
def get_list_input_before_interruption():
    entries = []
    try: 
        first_interruption_index = find(events, 'description', 'timeout for Research Task 1 triggered')
        first_interruption_time = int(events[first_interruption_index]['timestamp'])
       
        # find order list entries before that
        for event in events:
            if event['type'] == 'OrderListUserInput' and  int(event['timestamp']) < first_interruption_time:
                entries.append(event['description'])        
        return entries
    except TypeError: 
        return None
    


In [13]:
## performance
df_entries = pd.DataFrame(columns=['partnr', 'version', 'order list entries before interruption'])

directory = "C:\\Users\\janin\\Desktop\\Studium\\Masterarbeit\\03_Erhebung"
os.chdir(directory)
for file in os.listdir(directory):
    if file.endswith(".log"):
        with open(file, newline = '', encoding="utf-8") as stream:
            data_reader = csv.DictReader(stream, delimiter='\t')
            data_list = []
            for row in data_reader:
                data_list.append(dict(row))
                

            filename = os.path.splitext(file)[0]
        
            keys = list(data_list[1])
            key_1 = keys[0]
            key_2 = keys[1]
            key_3 = keys[2]

            for dictionary in data_list: 
                dictionary['type'] = dictionary.pop(key_2)
                dictionary['description'] = dictionary.pop(key_3)
                dictionary['timestamp'] = dictionary.pop(key_1)

            events = [{'type': key_2, 'description': key_3, 'timestamp': key_1}] + data_list

        # get useractions
            useractions = list(filter(lambda events: events['type'] == 'UserAction' or events['type'] == 'OrderListUserInput', events))
            
            # list entries before first interruption
            entries = get_list_input_before_interruption()
            
            df_e = pd.DataFrame([[entries]], columns = ['order list entries before interruption'])
            
        with open(filename + '.yaml', 'r') as stream:
        
            background_and_quest = yaml.safe_load(stream)
            userInputs = background_and_quest['userInputs']
            
            df_0 = pd.DataFrame([[int(background_and_quest['participantNumber']),  float(background_and_quest['version'])]], columns= ['partnr', 'version'])
            df_entries_0 = pd.concat([df_0, df_e], axis=1, sort=False)
        
            df_entries = df_entries.append(df_entries_0, ignore_index=True)
        
            

In [14]:
df_entries['order list entries before interruption'] = df_entries['order list entries before interruption'].astype(str)

a = df_entries.loc[df_entries['partnr'] == 67, 'order list entries before interruption'].item()


lst = a.split(",")
#or st in lst:
#    print(st)

In [15]:
# read csv file for performance (list entries)
directory = "C:\\Users\\janin\\Documents\\GitHub\\Masterthesis"

os.chdir(directory)

data = pd.read_csv('Auswertung_Leistung.csv', sep=';', encoding="ANSI") 

df_performance = data.iloc[:,[0,2,3,4, 7, 8, 9]]

df_performance.columns=["partnr", "nr_entries", "correct_entries", "wrong_entries", "entries_after_interruption", "correct_entries_after_interruption", "mistakes_after_interruption"]
df_performance.columns


Index(['partnr', 'nr_entries', 'correct_entries', 'wrong_entries',
       'entries_after_interruption', 'correct_entries_after_interruption',
       'mistakes_after_interruption'],
      dtype='object')

In [16]:
# create dataframe
df = pd.DataFrame(columns=['partnr', 'startTime', 'version',  'gender',  'age', 'termsOfStudies', 'techUsage', 'TAEG', 'TAEG_comments', 'NASATLX_1', 'NASATLX_2', 'NASATLX_3', 'NASATLX_4', 'purposefulactionlag1', 'purposeful_actionlag_2', 'purposeful_action_lag_3', 'resumption_lag_1', 'resumption_lag_2', 'resumption_lag_3', 'replacement_task_time', 'file', 'average_click_intervall', 'resumption_lag_1_alt_1', 'resumption_lag_2_alt_1', 'resumption_lag_3_alt_1', 'resumption_lag_1_alt_2', 'resumption_lag_2_alt_2', 'resumption_lag_3_alt_2', 'Technologienutzung: Arbeiten mit Office Programmen (Word, Excel, Powerpoint, etc.)', 'Technologienutzung: studienbezogene Aufgaben (z.B. Vorlesungen nachbereiten, Hausarbeiten schreiben, Literaturrecherche, etc.)', 'Technologienutzung: Email / im Internet surfen (z.B. Facebook, Spiegel Online, etc.)', 'Technologienutzung: Programmieren', 'Technologienutzung: Sonstiges'])

directory = "C:\\Users\\janin\\Desktop\\Studium\\Masterarbeit\\03_Erhebung"

os.chdir(directory)
for file in os.listdir(directory):
    if file.endswith(".log"):
        with open(file, newline = '', encoding="utf-8") as stream:
            data_reader = csv.DictReader(stream, delimiter='\t')
            data_list = []
            for row in data_reader:
                data_list.append(dict(row))
                

            filename = os.path.splitext(file)[0]
        
            print(filename)
        
            keys = list(data_list[1])
            print(keys)
            key_1 = keys[0]
            key_2 = keys[1]
            key_3 = keys[2]

            for dictionary in data_list: 
                dictionary['type'] = dictionary.pop(key_2)
                dictionary['description'] = dictionary.pop(key_3)
                dictionary['timestamp'] = dictionary.pop(key_1)

            events = [{'type': key_2, 'description': key_3, 'timestamp': key_1}] + data_list

        # get useractions
            useractions = list(filter(lambda events: events['type'] == 'UserAction' or events['type'] == 'OrderListUserInput', events))
        
        # time until purposeful action after interruption 1
            purposeful_action_lag_1 = compute_purposeful_action_lag('timeout for Wiki Task 1 triggered')

        # time until purposeful action after interruption 2
            purposeful_action_lag_2 = compute_purposeful_action_lag('timeout for Wiki Task 2 triggered')
            
        # time until purposeful action after interruption 3
            purposeful_action_lag_3 = compute_purposeful_action_lag('timeout for Wiki Task 3 triggered')

        # resumption lag 1
            resumption_lag_1 = compute_resumption_lag('timeout for Wiki Task 1 triggered')
        
        # resumption lag 2
            resumption_lag_2 = compute_resumption_lag('timeout for Wiki Task 2 triggered')
        
        # resumption lag 3
            resumption_lag_3 = compute_resumption_lag('timeout for Wiki Task 3 triggered')
            
            
                    # get average click intervall
            average_click_intervall = get_average_click_intervall()
            
        ## compute relative resumption lags (minus average_lag)
        # resumption lag 1
            resumption_lag_1_alt_1 = resumption_lag_1 - average_click_intervall 
        
        # resumption lag 2
            resumption_lag_2_alt_1 = resumption_lag_2 - average_click_intervall
        
        # resumption lag 3
            resumption_lag_3_alt_1 = resumption_lag_3 - average_click_intervall

                    ## compute relative resumption lags (divide by  average_lag)
        # resumption lag 1
            resumption_lag_1_alt_2 = resumption_lag_1 / average_click_intervall
        
        # resumption lag 2
            resumption_lag_2_alt_2 = resumption_lag_2 / average_click_intervall
        
        # resumption lag 3
            resumption_lag_3_alt_2 = resumption_lag_3 / average_click_intervall
            
            
        # time until order list input after interruption 1
            order_list_input_lag_1 = compute_order_list_time('timeout for Wiki Task 1 triggered')

        # time until order list input after interruption 2
            order_list_input_lag_2 = compute_order_list_time('timeout for Wiki Task 2 triggered')
            
        # time until order list input after interruption 3
            order_list_input_lag_3 = compute_order_list_time('timeout for Wiki Task 3 triggered')
        
        #replacement task
            replacement_task_time = get_replacement_task_time()
            

        
            dfEvents = pd.DataFrame([[purposeful_action_lag_1, purposeful_action_lag_2, purposeful_action_lag_3, resumption_lag_1, resumption_lag_2, resumption_lag_3, order_list_input_lag_1, order_list_input_lag_2, order_list_input_lag_3, replacement_task_time, filename, average_click_intervall, resumption_lag_1_alt_1, resumption_lag_2_alt_1, resumption_lag_3_alt_1, resumption_lag_1_alt_2, resumption_lag_2_alt_2, resumption_lag_3_alt_2]], columns=['purposeful_action_lag_1', 'purposeful_action_lag_2', 'purposeful_action_lag 3', 'resumption_lag_1', 'resumption_lag_2', 'resumption_lag_3', 'list_input_lag_1', 'list_input_lag_2', 'list_input_lag_3', 'replacement_task_time', 'file', 'average_click_intervall', 'resumption_lag_1_alt_1', 'resumption_lag_2_alt_1', 'resumption_lag_3_alt_1', 'resumption_lag_1_alt_2', 'resumption_lag_2_alt_2', 'resumption_lag_3_alt_2'] )

            

    
        with open(filename + '.yaml', 'r') as stream:
        
            background_and_quest = yaml.safe_load(stream)
            userInputs = background_and_quest['userInputs']
        # TA-EG mean
        
            taeg_results = compute_taeg(get_taeg_values())
            
        # TA-EG subscale enthusiasm
            taeg_enthusiasm = compute_taeg_subscale_enthusiasm(get_taeg_values())
            
        # TA-EG subscale competence
            taeg_competence = compute_taeg_subscale_competence(get_taeg_values())

        # TA-EG subscale positive attitude
            taeg_pos_attitude = compute_taeg_subscale_pos_attitude(get_taeg_values())
            
        # TA-EG subscale negative attitude
            taeg_neg_attitude = compute_taeg_subscale_neg_attitude(get_taeg_values())
            
            
        # TA-EG comments
            taeg_comments = get_taeg_comments()
        
        # NASA-TLX 1
            nasa_tlx_1_results = compute_quest_average(get_nasa_tlx_values('NASA TLX Questionnaire 1'))
        
        # NASA-TLX 2
            nasa_tlx_2_results = compute_quest_average(get_nasa_tlx_values('NASA TLX Questionnaire 2'))
                
        # NASA-TLX 3
            nasa_tlx_3_results = compute_quest_average(get_nasa_tlx_values('NASA TLX Questionnaire 3'))
        
        # NASA-TLX 4
            nasa_tlx_4_results = compute_quest_average(get_nasa_tlx_values('NASA TLX Questionnaire 4'))
        
        # SUS
            sus_results = compute_sus(get_sus_values())
        
        # tech usage answers
            tech_usage_msoffice = get_tech_usage_MC()[0]
            tech_usage_uni = get_tech_usage_MC()[1]
            tech_usage_internet = get_tech_usage_MC()[2]
            tech_usage_programming = get_tech_usage_MC()[3]
            tech_usage_others = get_tech_usage_MC_others()
        dfUserInput = pd.DataFrame([[int(background_and_quest['participantNumber']), float(background_and_quest['startTime']), float(background_and_quest['version']), str(background_and_quest['gender']), float(background_and_quest['age']), str(background_and_quest['termsOfStudies']), str(background_and_quest['techUsage']), taeg_results, taeg_enthusiasm, taeg_competence, taeg_pos_attitude, taeg_neg_attitude, taeg_comments, nasa_tlx_1_results, nasa_tlx_2_results, nasa_tlx_3_results, nasa_tlx_4_results, sus_results, tech_usage_msoffice, tech_usage_uni, tech_usage_internet, tech_usage_programming, tech_usage_others]], columns=['partnr', 'startTime','version',  'gender',  'age', 'termsOfStudies', 'techUsage', 'TAEG', 'taeg_enthusiasm', 'taeg_competence', 'taeg_pos_attitude', 'taeg_neg_attitude', 'TAEG_comments', 'NASATLX_1', 'NASATLX_2', 'NASATLX_3', 'NASATLX_4', 'SUS', 'Technologienutzung: Arbeiten mit Office Programmen (Word, Excel, Powerpoint, etc.)', 'Technologienutzung: studienbezogene Aufgaben (z.B. Vorlesungen nachbereiten, Hausarbeiten schreiben, Literaturrecherche, etc.)', 'Technologienutzung: Email / im Internet surfen (z.B. Facebook, Spiegel Online, etc.)', 'Technologienutzung: Programmieren', 'Technologienutzung: Sonstiges'] )
        dfall = pd.concat([dfUserInput,dfEvents], axis=1, sort=False)
        
        df = df.append(dfall, ignore_index=True)

Exp_No_11_at_2019-08-05_11-05-58-562
['1564995958729', 'System', 'now showing version selection']
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1564997747184'}
{'type': 'OrderListUserInput', 'description': 'user set amount of Maiskolben to "6"', 'timestamp': '1564997830596'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1564998344651'}
{'type': 'OrderListUserInput', 'description': 'user set vendor of Grillfleisch to "M"', 'timestamp': '1564998402865'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1564998815746'}
{'type': 'OrderListUserInput', 'description': 'user set vendor of Antipasti to "B"', 'timestamp': '1564998959504'}
38
3157.8947368421054
{'type': 'OrderListUserInput', 'description': 'user set amount of Maiskolben to "6"', 'timestamp': '1564997830596'}
{'type': 'OrderListUserInput', 'description': 'user set vendor o

Exp_No_21_at_2019-08-20_10-31-18-564
['1566289878632', 'System', 'now showing version selection']
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1566292341095'}
{'type': 'UserAction', 'description': 'user highlighted: Radler (bio)  15*0,5l  15,45 €', 'timestamp': '1566292387329'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1566292961088'}
{'type': 'UserAction', 'description': 'user removed note (ID=1566292556292) from webpage: http://localhost:8099/webcontent/exp4/Dateien/Getraenke/Getraenkemarkt_Mosel.html', 'timestamp': '1566292980788'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1566293447910'}
{'type': 'OrderListUserInput', 'description': 'user set amount of Grillkäse to "1"', 'timestamp': '1566293518024'}
26
4615.384615384615
No order list input during this task.
{'type': 'OrderListUserInput', 'description': 'user s

Exp_No_31_at_2019-08-06_16-29-07-899
['1565101747921', 'System', 'now showing version selection']
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565103224832'}
{'type': 'OrderListUserInput', 'description': 'user set costs of Antipasti to "1"', 'timestamp': '1565103373666'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565103817264'}
No meaningful action during this task.
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565104258120'}
No meaningful action during this task.
44
2727.2727272727275
{'type': 'OrderListUserInput', 'description': 'user set costs of Antipasti to "1"', 'timestamp': '1565103373666'}
No order list input during this task.
No order list input during this task.
Exp_No_32_at_2019-08-06_16-37-58-064
['1565102278087', 'System', 'now showing version selection']
{'type': 'OrderListFocus', 'description': 'user r

Exp_No_3_at_2019-08-15_10-35-53-247
['1565858153290', 'System', 'now showing version selection']
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565859993214'}
{'type': 'UserAction', 'description': 'user highlighted: Sorte  Anzahl  Preis  \xa0  Weizen  24*0,5l  17,30 €  \xa0  Radler (bio)  15*0,5l  15,45 €  \xa0  Regionales Bier (bio)  15*0,5l  15,00 €  \xa0  Malz (bio)  20*0,5l  9,95 €', 'timestamp': '1565860020375'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565860601487'}
{'type': 'UserAction', 'description': 'user added note to webpage (activity added, too): text=Bioradler 30€ gesamt, URL=http://localhost:8099/webcontent/exp4/Emails/Getraenkemarkt_Trier_Angebot.htm', 'timestamp': '1565860701601'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565861055418'}
{'type': 'OrderListUserInput', 'description': 'user set vend

Exp_No_47_at_2019-08-08_11-52-33-449
['1565257953463', 'System', 'now showing version selection']
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565260456036'}
{'type': 'OrderListUserInput', 'description': 'user set vendor of Bier to "G"', 'timestamp': '1565260494566'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565261036139'}
{'type': 'OrderListUserInput', 'description': 'user set vendor of Grillkäse to "/"', 'timestamp': '1565261129331'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565261496220'}
{'type': 'OrderListUserInput', 'description': 'user set costs of Grillfleisch to "4"', 'timestamp': '1565261502775'}
22
5454.545454545455
{'type': 'OrderListUserInput', 'description': 'user set vendor of Bier to "G"', 'timestamp': '1565260494566'}
{'type': 'OrderListUserInput', 'description': 'user set vendor of Grillkäse to

Exp_No_55_at_2019-08-09_14-53-42-226
['1565355222241', 'System', 'now showing version selection']
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565357688472'}
{'type': 'OrderListUserInput', 'description': 'user set amount of Salat (vegan) to "4"', 'timestamp': '1565357790940'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565358268569'}
{'type': 'OrderListUserInput', 'description': 'user set amount of Grillfleisch to "2"', 'timestamp': '1565358350912'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565358739187'}
{'type': 'OrderListUserInput', 'description': 'user set costs of Grillfleisch to "5"', 'timestamp': '1565358757162'}
9
13333.333333333334
{'type': 'OrderListUserInput', 'description': 'user set amount of Salat (vegan) to "4"', 'timestamp': '1565357790940'}
{'type': 'OrderListUserInput', 'description': 'user set a

Exp_No_65_at_2019-08-15_09-07-36-158
['1565852856357', 'System', 'now showing version selection']
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565854994350'}
{'type': 'OrderListUserInput', 'description': 'user set vendor of Mineralwasser to ""', 'timestamp': '1565855129719'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565855584556'}
{'type': 'UserAction', 'description': 'user removed note (ID=1565853591946) from webpage: http://localhost:8099/webcontent/exp4/Emails/Essen_bestellen_fuer_den_Grillabend.htm', 'timestamp': '1565855714870'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565856046766'}
{'type': 'UserAction', 'description': 'user added note to webpage (activity added, too): text=Grillkäse für 16, URL=http://localhost:8099/webcontent/exp4/Emails/Essen_bestellen_fuer_den_Grillabend.htm', 'timestamp': '156585605

In [17]:
# combine with df_performance
df = pd.merge(left=df,right=df_performance, left_on='partnr', right_on='partnr')
df

,NASATLX_1,NASATLX_2,NASATLX_3,NASATLX_4,SUS,TAEG,TAEG_comments,"Technologienutzung: Arbeiten mit Office Programmen (Word, Excel, Powerpoint, etc.)","Technologienutzung: Email / im Internet surfen (z.B. Facebook, Spiegel Online, etc.)",Technologienutzung: Programmieren,...,taeg_pos_attitude,techUsage,termsOfStudies,version,nr_entries,correct_entries,wrong_entries,entries_after_interruption,correct_entries_after_interruption,mistakes_after_interruption
0,5.333333,6.000000,6.000000,6.166667,27.5,3.368421,None,False,True,False,...,2.4,mehr als 6 Stunden tÃ¤glich,Erziehungswissenschaft,5.0,44,12,32,29,7,22
1,3.833333,5.000000,4.833333,4.833333,77.5,1.947368,None,True,False,False,...,2.0,mehr als 6 Stunden tÃ¤glich,Soziologie,6.0,13,11,2,10,8,2
2,2.166667,2.833333,3.000000,2.500000,40.0,1.526316,None,True,True,False,...,1.2,zwischen 2 und 6 Stunden tÃ¤glich,Psychologie,1.0,34,28,6,18,14,4
3,3.333333,4.000000,4.500000,4.333333,17.5,3.631579,None,False,True,False,...,2.6,zwischen einer halben und 2 Stunden tÃ¤glich,Psychologie,3.0,19,12,7,7,2,5
4,3.833333,3.500000,4.500000,4.166667,50.0,2.736842,None,False,False,False,...,2.2,zwischen 2 und 6 Stunden tÃ¤glich,Erziehungswissenschaften,4.0,29,19,5,9,7,2
5,5.166667,5.333333,5.666667,6.500000,35.0,2.842105,None,False,False,False,...,2.2,zwischen 2 und 6 Stunden tÃ¤glich,Psychologie,6.0,15,11,4,8,5,3
6,4.166667,4.333333,3.666667,3.500000,60.0,2.473684,None,True,True,False,...,2.2,zwischen einer halben und 2 Stunden tÃ¤glich,SozialpÃ¤dagogik,1.0,38,19,19,22,11,11
7,3.166667,3.833333,4.500000,4.333333,65.0,3.684211,None,True,True,False,...,2.4,zwischen 2 und 6 Stunden tÃ¤glich,Psychologie,2.0,28,25,3,12,10,2
8,3.833333,4.166667,4.333333,4.500000,62.5,3.526316,None,False,True,False,...,2.4,zwischen 2 und 6 Stunden tÃ¤glich,Germanistik,3.0,14,13,1,5,4,1
9,5.833333,6.500000,4.666667,5.333333,20.0,2.421053,None,True,True,False,...,2.2,zwischen 2 und 6 Stunden tÃ¤glich,Anglistik,5.0,33,20,13,21,8,13


In [18]:
# df 2: Task nr and version as factors

df_messwdh = pd.DataFrame(columns=['TAEG', 'trial', 'NASATLX',  'purposeful_action_lags', 'resumption_lags', 'list_input_lags', 'average_click_intervall', 'resumption_lags_alt1', 'resumption_lags_alt2'])
df_nasa = pd.DataFrame(columns=['TAEG', 'trial', 'NASATLX',  'purposeful_action_lags', 'resumption_lags', 'list_input_lags', 'average_click_intervall', 'resumption_lags_alt1', 'resumption_lags_alt2'])


purposeful_action_lags = []
resumption_lags = []
nasa_tlx_results = []

directory = "C:\\Users\\janin\\Desktop\\Studium\\Masterarbeit\\03_Erhebung"
os.chdir(directory)
for file in os.listdir(directory):
    if file.endswith(".log"):
        with open(file, newline = '', encoding="utf-8") as stream:
            data_reader = csv.DictReader(stream, delimiter='\t')
            data_list = []
            for row in data_reader:
                data_list.append(dict(row))
                

            filename = os.path.splitext(file)[0]
        
            print(filename)
        
            keys = list(data_list[1])
            print(keys)
            key_1 = keys[0]
            key_2 = keys[1]
            key_3 = keys[2]

            for dictionary in data_list: 
                dictionary['type'] = dictionary.pop(key_2)
                dictionary['description'] = dictionary.pop(key_3)
                dictionary['timestamp'] = dictionary.pop(key_1)

            events = [{'type': key_2, 'description': key_3, 'timestamp': key_1}] + data_list

        # get useractions
            useractions = list(filter(lambda events: events['type'] == 'UserAction' or events['type'] == 'OrderListUserInput', events))
        
        # time until purposeful action after interruption 1
            purposeful_action_lag_1 = compute_purposeful_action_lag('timeout for Wiki Task 1 triggered')

        # time until purposeful action after interruption 2
            purposeful_action_lag_2 = compute_purposeful_action_lag('timeout for Wiki Task 2 triggered')
            
        # time until purposeful action after interruption 3
            purposeful_action_lag_3 = compute_purposeful_action_lag('timeout for Wiki Task 3 triggered')
        

        # resumption lag 1
            resumption_lag_1 = compute_resumption_lag('timeout for Wiki Task 1 triggered')
        
        # resumption lag 2
            resumption_lag_2 = compute_resumption_lag('timeout for Wiki Task 2 triggered')
        
        # resumption lag 3
            resumption_lag_3 = compute_resumption_lag('timeout for Wiki Task 3 triggered')
 

        # get average click intervall
            average_click_intervall = get_average_click_intervall()
        
        
            
          # resumption lag 1
            resumption_lag_1 = compute_resumption_lag('timeout for Wiki Task 1 triggered')
        
        # resumption lag 2
            resumption_lag_2 = compute_resumption_lag('timeout for Wiki Task 2 triggered')
        
        # resumption lag 3
            resumption_lag_3 = compute_resumption_lag('timeout for Wiki Task 3 triggered')
            
            
        # resumption lag 1
            resumption_lag_1_alt_1 = average_click_intervall - resumption_lag_1 
        
        # resumption lag 2
            resumption_lag_2_alt_1 = average_click_intervall - resumption_lag_2 
        
        # resumption lag 3
            resumption_lag_3_alt_1 = average_click_intervall - resumption_lag_3

                    ## compute relative resumption lags (divide by  average_lag)
        # resumption lag 1
            resumption_lag_1_alt_2 = resumption_lag_1 / average_click_intervall
        
        # resumption lag 2
            resumption_lag_2_alt_2 = resumption_lag_2 / average_click_intervall
        
        # resumption lag 3
            resumption_lag_3_alt_2 = resumption_lag_3 / average_click_intervall
           
            
        # time until order list input after interruption 1
            order_list_input_lag_1 = compute_order_list_time('timeout for Wiki Task 1 triggered')

        # time until order list input after interruption 2
            order_list_input_lag_2 = compute_order_list_time('timeout for Wiki Task 2 triggered')
            
        # time until order list input after interruption 3
            order_list_input_lag_3 = compute_order_list_time('timeout for Wiki Task 3 triggered')
        

        
        df_messung_1 = pd.DataFrame([[1, purposeful_action_lag_1, resumption_lag_1, order_list_input_lag_1, resumption_lag_1_alt_1, resumption_lag_1_alt_2, average_click_intervall]], columns=['trial', 'purposeful_action_lags', 'resumption_lags', 'list_input_lags', 'resumption_lags_alt1', 'resumption_lags_alt2', 'average_click_intervall'])
        df_messung_2 = pd.DataFrame([[2, purposeful_action_lag_2, resumption_lag_2, order_list_input_lag_2, resumption_lag_2_alt_1, resumption_lag_2_alt_2, average_click_intervall]],columns=['trial', 'purposeful_action_lags', 'resumption_lags', 'list_input_lags', 'resumption_lags_alt1', 'resumption_lags_alt2', 'average_click_intervall'])
        df_messung_3 = pd.DataFrame([[3, purposeful_action_lag_3,resumption_lag_3, order_list_input_lag_3, resumption_lag_3_alt_1, resumption_lag_3_alt_2, average_click_intervall]],columns=['trial', 'purposeful_action_lags', 'resumption_lags', 'list_input_lags', 'resumption_lags_alt1', 'resumption_lags_alt2', 'average_click_intervall'])
        df_messung_0 = pd.DataFrame([[0, 0, 0, 0, 0, 0, 0]], columns=['trial', 'purposeful_action_lags', 'resumption_lags', 'list_input_lags', 'resumption_lags_alt1', 'resumption_lags_alt2',  'average_click_intervall'])
 
            
 
    
        with open(filename + '.yaml', 'r') as stream:
        
            background_and_quest = yaml.safe_load(stream)
            userInputs = background_and_quest['userInputs']
        
        # TA-EG values
        
        
            taeg_results = compute_taeg(get_taeg_values())
        
        
        # NASA-TLX 1 (Baseline)
            nasa_tlx_0_results = compute_quest_average(get_nasa_tlx_values('NASA TLX Questionnaire 1'))
            nasa_tlx_mental_demand_0 = int(get_nasa_tlx_values('NASA TLX Questionnaire 1')[0]['receivedAnswer'])
            nasa_tlx_physical_demand_0 = int(get_nasa_tlx_values('NASA TLX Questionnaire 1')[1]['receivedAnswer'])
            nasa_tlx_time_0 = int(get_nasa_tlx_values('NASA TLX Questionnaire 1')[2]['receivedAnswer'])
            nasa_tlx_performance_0 = int(get_nasa_tlx_values('NASA TLX Questionnaire 1')[3]['receivedAnswer'])
            nasa_tlx_effort_0 = int(get_nasa_tlx_values('NASA TLX Questionnaire 1')[4]['receivedAnswer'])
            nasa_tlx_frustration_0 = int(get_nasa_tlx_values('NASA TLX Questionnaire 1')[5]['receivedAnswer'])
        
        # NASA-TLX 2
            nasa_tlx_1_results = compute_quest_average(get_nasa_tlx_values('NASA TLX Questionnaire 2'))
            nasa_tlx_mental_demand_1 = int(get_nasa_tlx_values('NASA TLX Questionnaire 2')[0]['receivedAnswer'])
            nasa_tlx_physical_demand_1 = int(get_nasa_tlx_values('NASA TLX Questionnaire 2')[1]['receivedAnswer'])
            nasa_tlx_time_1 = int(get_nasa_tlx_values('NASA TLX Questionnaire 2')[2]['receivedAnswer'])
            nasa_tlx_performance_1 = int(get_nasa_tlx_values('NASA TLX Questionnaire 2')[3]['receivedAnswer'])
            nasa_tlx_effort_1 = int(get_nasa_tlx_values('NASA TLX Questionnaire 2')[4]['receivedAnswer'])
            nasa_tlx_frustration_1 = int(get_nasa_tlx_values('NASA TLX Questionnaire 2')[5]['receivedAnswer'])
                
        # NASA-TLX 3
            nasa_tlx_2_results = compute_quest_average(get_nasa_tlx_values('NASA TLX Questionnaire 3'))
            nasa_tlx_mental_demand_2 = int(get_nasa_tlx_values('NASA TLX Questionnaire 3')[0]['receivedAnswer'])
            nasa_tlx_physical_demand_2 = int(get_nasa_tlx_values('NASA TLX Questionnaire 3')[1]['receivedAnswer'])
            nasa_tlx_time_2 = int(get_nasa_tlx_values('NASA TLX Questionnaire 3')[2]['receivedAnswer'])
            nasa_tlx_performance_2 = int(get_nasa_tlx_values('NASA TLX Questionnaire 3')[3]['receivedAnswer'])
            nasa_tlx_effort_2 = int(get_nasa_tlx_values('NASA TLX Questionnaire 3')[4]['receivedAnswer'])
            nasa_tlx_frustration_2 =  int(get_nasa_tlx_values('NASA TLX Questionnaire 3')[5]['receivedAnswer'])
            
            
        # NASA-TLX 4
            nasa_tlx_3_results = compute_quest_average(get_nasa_tlx_values('NASA TLX Questionnaire 4'))
            nasa_tlx_mental_demand_3 =  int(get_nasa_tlx_values('NASA TLX Questionnaire 4')[0]['receivedAnswer'])
            nasa_tlx_physical_demand_3 =  int(get_nasa_tlx_values('NASA TLX Questionnaire 4')[1]['receivedAnswer'])
            nasa_tlx_time_3 =  int(get_nasa_tlx_values('NASA TLX Questionnaire 4')[2]['receivedAnswer'])
            nasa_tlx_performance_3 =  int(get_nasa_tlx_values('NASA TLX Questionnaire 4')[3]['receivedAnswer'])
            nasa_tlx_effort_3 =  int(get_nasa_tlx_values('NASA TLX Questionnaire 4')[4]['receivedAnswer'])
            nasa_tlx_frustration_3 =  int(get_nasa_tlx_values('NASA TLX Questionnaire 4')[5]['receivedAnswer'])

        
        # create dataframe
        dfUserInput_messwdh_0 = pd.DataFrame([[int(background_and_quest['participantNumber']),  float(background_and_quest['version']),  taeg_results, taeg_enthusiasm, taeg_competence, taeg_pos_attitude, taeg_neg_attitude, nasa_tlx_0_results, nasa_tlx_mental_demand_0, nasa_tlx_physical_demand_0, nasa_tlx_time_0, nasa_tlx_performance_0, nasa_tlx_effort_0, nasa_tlx_frustration_0]], columns=['partnr', 'version',   'TAEG',  'taeg_enthusiasm', 'taeg_competence', 'taeg_pos_attitude', 'taeg_neg_attitude', 'NASATLX', 'nasa_tlx_mental_demand', 'nasa_tlx_physical_demand', 'nasa_tlx_time', 'nasa_tlx_performance', 'nasa_tlx_effort', 'nasa_tlx_frustration'] )
        dfUserInput_messwdh_1 = pd.DataFrame([[int(background_and_quest['participantNumber']),  float(background_and_quest['version']),  taeg_results, taeg_enthusiasm, taeg_competence, taeg_pos_attitude, taeg_neg_attitude, nasa_tlx_1_results, nasa_tlx_mental_demand_1, nasa_tlx_physical_demand_1, nasa_tlx_time_1, nasa_tlx_performance_1, nasa_tlx_effort_1, nasa_tlx_frustration_1]], columns=['partnr', 'version',   'TAEG',  'taeg_enthusiasm', 'taeg_competence', 'taeg_pos_attitude', 'taeg_neg_attitude', 'NASATLX', 'nasa_tlx_mental_demand', 'nasa_tlx_physical_demand', 'nasa_tlx_time', 'nasa_tlx_performance', 'nasa_tlx_effort', 'nasa_tlx_frustration'] )
        dfUserInput_messwdh_2 = pd.DataFrame([[int(background_and_quest['participantNumber']),  float(background_and_quest['version']),  taeg_results, taeg_enthusiasm, taeg_competence, taeg_pos_attitude, taeg_neg_attitude, nasa_tlx_2_results, nasa_tlx_mental_demand_2, nasa_tlx_physical_demand_2, nasa_tlx_time_2, nasa_tlx_performance_2, nasa_tlx_effort_2, nasa_tlx_frustration_2]], columns=['partnr', 'version',   'TAEG',  'taeg_enthusiasm', 'taeg_competence', 'taeg_pos_attitude', 'taeg_neg_attitude', 'NASATLX', 'nasa_tlx_mental_demand', 'nasa_tlx_physical_demand', 'nasa_tlx_time', 'nasa_tlx_performance', 'nasa_tlx_effort', 'nasa_tlx_frustration'] )
        dfUserInput_messwdh_3 = pd.DataFrame([[int(background_and_quest['participantNumber']),  float(background_and_quest['version']),  taeg_results, taeg_enthusiasm, taeg_competence, taeg_pos_attitude, taeg_neg_attitude, nasa_tlx_3_results, nasa_tlx_mental_demand_3, nasa_tlx_physical_demand_3, nasa_tlx_time_3, nasa_tlx_performance_3, nasa_tlx_effort_3, nasa_tlx_frustration_3]], columns=['partnr', 'version',   'TAEG',  'taeg_enthusiasm', 'taeg_competence', 'taeg_pos_attitude', 'taeg_neg_attitude', 'NASATLX', 'nasa_tlx_mental_demand', 'nasa_tlx_physical_demand', 'nasa_tlx_time', 'nasa_tlx_performance', 'nasa_tlx_effort', 'nasa_tlx_frustration'] )
        
        df_1 = pd.concat([df_messung_1, dfUserInput_messwdh_1], axis=1, sort=False)
        df_2 = pd.concat([df_messung_2, dfUserInput_messwdh_2], axis=1, sort=False)
        df_3 = pd.concat([df_messung_3, dfUserInput_messwdh_3], axis=1, sort=False)
        df_0 = pd.concat([df_messung_0, dfUserInput_messwdh_0], axis=1, sort=False)

        
        df_all_2 = pd.concat([df_1, df_2, df_3], axis=0, sort=False)
        df_all_0 = pd.concat([df_0, df_1, df_2, df_3], axis=0, sort=False)
        
        
        df_messwdh = df_messwdh.append(df_all_2, ignore_index=True)
        df_nasa = df_nasa.append(df_all_0, ignore_index=True)
        


Exp_No_11_at_2019-08-05_11-05-58-562
['1564995958729', 'System', 'now showing version selection']
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1564997747184'}
{'type': 'OrderListUserInput', 'description': 'user set amount of Maiskolben to "6"', 'timestamp': '1564997830596'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1564998344651'}
{'type': 'OrderListUserInput', 'description': 'user set vendor of Grillfleisch to "M"', 'timestamp': '1564998402865'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1564998815746'}
{'type': 'OrderListUserInput', 'description': 'user set vendor of Antipasti to "B"', 'timestamp': '1564998959504'}
38
3157.8947368421054
{'type': 'OrderListUserInput', 'description': 'user set amount of Maiskolben to "6"', 'timestamp': '1564997830596'}
{'type': 'OrderListUserInput', 'description': 'user set vendor o

Exp_No_20_at_2019-08-05_17-06-01-600
['1565017561624', 'System', 'now showing version selection']
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565018886324'}
{'type': 'OrderListUserInput', 'description': 'user set costs of Saft to "2"', 'timestamp': '1565019024867'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565019457627'}
{'type': 'OrderListUserInput', 'description': 'user set amount of Grillkäse to "1"', 'timestamp': '1565019590133'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565019903631'}
{'type': 'OrderListUserInput', 'description': 'user set vendor of Grillkäse to "Königs Demetermar"', 'timestamp': '1565019945728'}
7
17142.85714285714
{'type': 'OrderListUserInput', 'description': 'user set costs of Saft to "2"', 'timestamp': '1565019024867'}
{'type': 'OrderListUserInput', 'description': 'user set amount of G

Exp_No_28_at_2019-08-21_13-26-08-477
['1566386768521', 'System', 'now showing version selection']
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1566389372623'}
{'type': 'OrderListUserInput', 'description': 'user set vendor of Bier to "Get"', 'timestamp': '1566389377828'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1566389974155'}
{'type': 'UserAction', 'description': 'user removed note (ID=1566387786980) from webpage: http://localhost:8099/webcontent/exp4/Emails/Essen_bestellen_fuer_den_Grillabend.htm', 'timestamp': '1566390069110'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1566390425074'}
No meaningful action during this task.
11
10909.09090909091
{'type': 'OrderListUserInput', 'description': 'user set vendor of Bier to "Get"', 'timestamp': '1566389377828'}
{'type': 'OrderListUserInput', 'description': 'user set amou

Exp_No_37_at_2019-08-07_09-05-11-755
['1565161511804', 'System', 'now showing version selection']
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565164309403'}
{'type': 'OrderListUserInput', 'description': 'user set amount of Grillkäse to "7"', 'timestamp': '1565164348663'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565164919338'}
{'type': 'OrderListUserInput', 'description': 'user set costs of Grillkäse to "13,23€"', 'timestamp': '1565164925377'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565165382581'}
{'type': 'OrderListUserInput', 'description': 'user set amount of Grillfleisch to "2"', 'timestamp': '1565165433799'}
8
15000.0
{'type': 'OrderListUserInput', 'description': 'user set amount of Grillkäse to "7"', 'timestamp': '1565164348663'}
{'type': 'OrderListUserInput', 'description': 'user set costs of Grillkäse

{'type': 'OrderListUserInput', 'description': 'user set vendor of Grillfleisch to "m"', 'timestamp': '1565260225344'}
Exp_No_44_at_2019-08-08_11-54-25-510
['1565258065557', 'System', 'now showing version selection']
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565259748638'}
{'type': 'OrderListUserInput', 'description': 'user set amount of Saft to "15"', 'timestamp': '1565259779790'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565260323512'}
{'type': 'OrderListUserInput', 'description': 'user set amount of Grillsauce to "4("', 'timestamp': '1565260392743'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565260773693'}
{'type': 'OrderListUserInput', 'description': 'user set amount of Bier to "6(30Flaschen?)"', 'timestamp': '1565260809656'}
6
20000.0
{'type': 'OrderListUserInput', 'description': 'user set amount of Saft t

Exp_No_53_at_2019-08-09_09-00-54-310
['1565334054397', 'System', 'now showing version selection']
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565336158600'}
{'type': 'OrderListUserInput', 'description': 'user set costs of Bier to "3"', 'timestamp': '1565336195016'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565336771110'}
{'type': 'UserAction', 'description': 'user highlighted: Maiskolben  3 € / 2 Stück  inkl. MwSt.', 'timestamp': '1565336875314'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565337256031'}
{'type': 'OrderListUserInput', 'description': 'user set vendor of Paprika to "Regio-gemüsesupermarkt"', 'timestamp': '1565337264924'}
38
3157.8947368421054
{'type': 'OrderListUserInput', 'description': 'user set costs of Bier to "3"', 'timestamp': '1565336195016'}
{'type': 'OrderListUserInput', 'description': 'us

Exp_No_62_at_2019-08-14_10-34-33-527
['1565771673578', 'System', 'now showing version selection']
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565773262020'}
{'type': 'OrderListUserInput', 'description': 'user set vendor of Wein to "M"', 'timestamp': '1565773286246'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565773832799'}
{'type': 'OrderListUserInput', 'description': 'user set amount of Aubergine to "4"', 'timestamp': '1565773872098'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1565774273248'}
{'type': 'OrderListUserInput', 'description': 'user set costs of Zucchini to "4"', 'timestamp': '1565774308883'}
70
1714.2857142857142
{'type': 'OrderListUserInput', 'description': 'user set vendor of Wein to "M"', 'timestamp': '1565773286246'}
{'type': 'OrderListUserInput', 'description': 'user set amount of Aubergine to "4

Exp_No_7_at_2019-08-05_11-01-18-704
['1564995679027', 'System', 'now showing version selection']
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1564997397666'}
{'type': 'OrderListUserInput', 'description': 'user set costs of Grillkäse to "3"', 'timestamp': '1564997574560'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1564997998630'}
{'type': 'OrderListUserInput', 'description': 'user set amount of Grillfleisch to "1"', 'timestamp': '1564998208148'}
{'type': 'OrderListFocus', 'description': 'user returned to order list (sync called)', 'timestamp': '1564998451233'}
{'type': 'OrderListUserInput', 'description': 'user set vendor of Bratwürste to "Metzgerei Biobio"', 'timestamp': '1564998633214'}
27
4444.444444444444
{'type': 'OrderListUserInput', 'description': 'user set costs of Grillkäse to "3"', 'timestamp': '1564997574560'}
{'type': 'OrderListUserInput', 'description': 'user se

In [19]:
df

,NASATLX_1,NASATLX_2,NASATLX_3,NASATLX_4,SUS,TAEG,TAEG_comments,"Technologienutzung: Arbeiten mit Office Programmen (Word, Excel, Powerpoint, etc.)","Technologienutzung: Email / im Internet surfen (z.B. Facebook, Spiegel Online, etc.)",Technologienutzung: Programmieren,...,taeg_pos_attitude,techUsage,termsOfStudies,version,nr_entries,correct_entries,wrong_entries,entries_after_interruption,correct_entries_after_interruption,mistakes_after_interruption
0,5.333333,6.000000,6.000000,6.166667,27.5,3.368421,None,False,True,False,...,2.4,mehr als 6 Stunden tÃ¤glich,Erziehungswissenschaft,5.0,44,12,32,29,7,22
1,3.833333,5.000000,4.833333,4.833333,77.5,1.947368,None,True,False,False,...,2.0,mehr als 6 Stunden tÃ¤glich,Soziologie,6.0,13,11,2,10,8,2
2,2.166667,2.833333,3.000000,2.500000,40.0,1.526316,None,True,True,False,...,1.2,zwischen 2 und 6 Stunden tÃ¤glich,Psychologie,1.0,34,28,6,18,14,4
3,3.333333,4.000000,4.500000,4.333333,17.5,3.631579,None,False,True,False,...,2.6,zwischen einer halben und 2 Stunden tÃ¤glich,Psychologie,3.0,19,12,7,7,2,5
4,3.833333,3.500000,4.500000,4.166667,50.0,2.736842,None,False,False,False,...,2.2,zwischen 2 und 6 Stunden tÃ¤glich,Erziehungswissenschaften,4.0,29,19,5,9,7,2
5,5.166667,5.333333,5.666667,6.500000,35.0,2.842105,None,False,False,False,...,2.2,zwischen 2 und 6 Stunden tÃ¤glich,Psychologie,6.0,15,11,4,8,5,3
6,4.166667,4.333333,3.666667,3.500000,60.0,2.473684,None,True,True,False,...,2.2,zwischen einer halben und 2 Stunden tÃ¤glich,SozialpÃ¤dagogik,1.0,38,19,19,22,11,11
7,3.166667,3.833333,4.500000,4.333333,65.0,3.684211,None,True,True,False,...,2.4,zwischen 2 und 6 Stunden tÃ¤glich,Psychologie,2.0,28,25,3,12,10,2
8,3.833333,4.166667,4.333333,4.500000,62.5,3.526316,None,False,True,False,...,2.4,zwischen 2 und 6 Stunden tÃ¤glich,Germanistik,3.0,14,13,1,5,4,1
9,5.833333,6.500000,4.666667,5.333333,20.0,2.421053,None,True,True,False,...,2.2,zwischen 2 und 6 Stunden tÃ¤glich,Anglistik,5.0,33,20,13,21,8,13


In [20]:
df_nasa

,NASATLX,TAEG,average_click_intervall,list_input_lags,nasa_tlx_effort,nasa_tlx_frustration,nasa_tlx_mental_demand,nasa_tlx_performance,nasa_tlx_physical_demand,nasa_tlx_time,...,purposeful_action_lags,resumption_lags,resumption_lags_alt1,resumption_lags_alt2,taeg_competence,taeg_enthusiasm,taeg_neg_attitude,taeg_pos_attitude,trial,version
0,5.333333,3.368421,0.000000,0,6.0,6.0,6.0,2.0,5.0,7.0,...,0,0,0.000000,0.000000,2.25,2.0,3.4,1.4,0,5.0
1,6.000000,3.368421,3157.894737,83412,5.0,7.0,5.0,7.0,5.0,7.0,...,83412,5905,-2747.105263,1.869917,2.25,2.0,3.4,1.4,1,5.0
2,6.000000,3.368421,3157.894737,58214,5.0,7.0,5.0,7.0,5.0,7.0,...,58214,8793,-5635.105263,2.784450,2.25,2.0,3.4,1.4,2,5.0
3,6.166667,3.368421,3157.894737,143758,5.0,7.0,6.0,7.0,5.0,7.0,...,143758,17513,-14355.105263,5.545783,2.25,2.0,3.4,1.4,3,5.0
4,3.833333,1.947368,0.000000,0,4.0,3.0,6.0,5.0,2.0,3.0,...,0,0,0.000000,0.000000,2.25,2.0,3.4,1.4,0,6.0
5,5.000000,1.947368,60000.000000,8333,6.0,6.0,6.0,6.0,1.0,5.0,...,8333,8333,51667.000000,0.138883,2.25,2.0,3.4,1.4,1,6.0
6,4.833333,1.947368,60000.000000,6098,5.0,5.0,5.0,6.0,1.0,7.0,...,6098,6098,53902.000000,0.101633,2.25,2.0,3.4,1.4,2,6.0
7,4.833333,1.947368,60000.000000,97236,6.0,3.0,6.0,7.0,1.0,6.0,...,78884,78884,-18884.000000,1.314733,2.25,2.0,3.4,1.4,3,6.0
8,2.166667,1.526316,0.000000,0,2.0,1.0,3.0,4.0,1.0,2.0,...,0,0,0.000000,0.000000,2.25,2.0,3.4,1.4,0,1.0
9,2.833333,1.526316,8571.428571,154013,2.0,3.0,3.0,3.0,3.0,3.0,...,77227,4465,4106.428571,0.520917,2.25,2.0,3.4,1.4,1,1.0


In [21]:
df_messwdh

,NASATLX,TAEG,average_click_intervall,list_input_lags,nasa_tlx_effort,nasa_tlx_frustration,nasa_tlx_mental_demand,nasa_tlx_performance,nasa_tlx_physical_demand,nasa_tlx_time,...,purposeful_action_lags,resumption_lags,resumption_lags_alt1,resumption_lags_alt2,taeg_competence,taeg_enthusiasm,taeg_neg_attitude,taeg_pos_attitude,trial,version
0,6.000000,3.368421,3157.894737,83412,5.0,7.0,5.0,7.0,5.0,7.0,...,83412,5905,-2747.105263,1.869917,2.25,2.0,3.4,1.4,1,5.0
1,6.000000,3.368421,3157.894737,58214,5.0,7.0,5.0,7.0,5.0,7.0,...,58214,8793,-5635.105263,2.784450,2.25,2.0,3.4,1.4,2,5.0
2,6.166667,3.368421,3157.894737,143758,5.0,7.0,6.0,7.0,5.0,7.0,...,143758,17513,-14355.105263,5.545783,2.25,2.0,3.4,1.4,3,5.0
3,5.000000,1.947368,60000.000000,8333,6.0,6.0,6.0,6.0,1.0,5.0,...,8333,8333,51667.000000,0.138883,2.25,2.0,3.4,1.4,1,6.0
4,4.833333,1.947368,60000.000000,6098,5.0,5.0,5.0,6.0,1.0,7.0,...,6098,6098,53902.000000,0.101633,2.25,2.0,3.4,1.4,2,6.0
5,4.833333,1.947368,60000.000000,97236,6.0,3.0,6.0,7.0,1.0,6.0,...,78884,78884,-18884.000000,1.314733,2.25,2.0,3.4,1.4,3,6.0
6,2.833333,1.526316,8571.428571,154013,2.0,3.0,3.0,3.0,3.0,3.0,...,77227,4465,4106.428571,0.520917,2.25,2.0,3.4,1.4,1,1.0
7,3.000000,1.526316,8571.428571,30221,3.0,3.0,2.0,4.0,3.0,3.0,...,30221,2935,5636.428571,0.342417,2.25,2.0,3.4,1.4,2,1.0
8,2.500000,1.526316,8571.428571,51883,2.0,3.0,2.0,4.0,2.0,2.0,...,51883,8754,-182.571429,1.021300,2.25,2.0,3.4,1.4,3,1.0
9,4.000000,3.631579,2000.000000,5021,4.0,5.0,4.0,5.0,1.0,5.0,...,5021,5021,-3021.000000,2.510500,2.25,2.0,3.4,1.4,1,3.0


In [22]:
df.groupby('version').count()





,NASATLX_1,NASATLX_2,NASATLX_3,NASATLX_4,SUS,TAEG,TAEG_comments,"Technologienutzung: Arbeiten mit Office Programmen (Word, Excel, Powerpoint, etc.)","Technologienutzung: Email / im Internet surfen (z.B. Facebook, Spiegel Online, etc.)",Technologienutzung: Programmieren,...,taeg_neg_attitude,taeg_pos_attitude,techUsage,termsOfStudies,nr_entries,correct_entries,wrong_entries,entries_after_interruption,correct_entries_after_interruption,mistakes_after_interruption
version,,,,,,,,,,,,,,,,,,,,,
1.0,11,11,11,11,11,11,0,11,11,11,...,11,11,11,11,11,11,11,11,11,11
2.0,10,10,10,10,10,10,0,10,10,10,...,10,10,10,10,10,10,10,10,10,10
3.0,6,6,6,6,6,6,0,6,6,6,...,6,6,6,6,6,6,6,6,6,6
4.0,7,7,7,7,7,7,0,7,7,7,...,7,7,7,7,7,7,7,7,7,7
5.0,10,10,10,10,10,10,0,10,10,10,...,10,10,10,10,10,10,10,10,10,10
6.0,11,11,11,11,11,11,0,11,11,11,...,11,11,11,11,11,11,11,11,11,11


In [24]:
# write to file


#feather.write_dataframe(df, 'C:\\Users\\janin\\Desktop\\Masterarbeit\\09_results_files\\results_test_18-03-2020.file')

#feather.write_dataframe(df_messwdh, 'C:\\Users\\janin\\Desktop\\Masterarbeit\\09_results_files\\results_test_18-03-2020_messwdh.file')

#feather.write_dataframe(df_nasa, 'C:\\Users\\janin\\Desktop\\Masterarbeit\\09_results_files\\results_test_18-03-2020_messwdh_nasa.file')

#feather.write_dataframe(df_entries, 'C:\\Users\\janin\\Desktop\\Masterarbeit\\09_results_files\\results_test_18-03-2020_list_input.file')



In [25]:
#feather.write_dataframe(df_useractions, 'C:\\Users\\janin\\Desktop\\Studium\\Masterarbeit\\09_results_files\\results_test_02-04-2020_user_actions.file')


['bd.', 'abc.']